In [1]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [2]:
with open('../data/training_data.pkl', 'rb') as f:
    train_data = pickle.load(f)

with open('../data/test_data.pkl', 'rb') as f:
    test_data = pickle.load(f)

with open('../data/training_labels.pkl', 'rb') as f:
    train_labels = pickle.load(f)

In [3]:
class knn:
        
        def __init__(self, kernel, k=3) -> None:
            self.k = k
            self.kernel = kernel
            
        def fit(self, X, y):
            self.X = X
            self.y = y

            return self
        
        def compute_distance(self, x, base):
            k = self.kernel(x,x)
            return [k+self.kernel(b,b)-2*self.kernel(x, b) for b in base]
        
        def predict(self, X):
            y_pred = []
            score_list = []
            for x in tqdm(X):
                scores = np.array(self.compute_distance(x, self.X))
                idx = np.argsort(scores)[:self.k]
                y_pred.append((self.y[idx].mean()>.5).astype(int))
                score_list.append(scores)
            return np.array(y_pred), np.array(score_list)
        
        def score(self, X, y):
            return np.mean(self.predict(X)[0]==y)

In [4]:
import sys 
import os

if 'src' not in [el.split('/')[-1] for el in sys.path]:
    curr_path = os.getcwd()
    sys.path.append('/'.join((curr_path.split('/')[:-1])))

In [5]:
from src.kernels import RandomWalkKernel, QuadraticEdgeKernel, VertexHistogramKernel

In [6]:
idx1 = train_labels==1
idx0 = train_labels==0

train_data = np.array(train_data, dtype=object)

X_train = train_data[idx1][:30]
X_train = np.concatenate((X_train, train_data[idx0][:30]))
y_train = np.concatenate((np.ones(30), np.zeros(30)))

X_test = train_data[idx1][30:60]
X_test = np.concatenate((X_test, train_data[idx0][30:60]))
y_test = np.concatenate((np.ones(30), np.zeros(30)))



#knn_ = knn(k=3, kernel=RandomWalkKernel(edge_kernel=QuadraticEdgeKernel())).fit(X_train, y_train)
knn_ = knn(k=3, kernel=VertexHistogramKernel()).fit(X_train, y_train)


In [7]:
a = knn_.predict(X_test)
np.mean(a[0]==y_test)

100%|██████████| 60/60 [00:00<00:00, 72.82it/s]


0.5333333333333333